In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 29.1 MB/s eta 0:00:00


In [1]:
pip install ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.8 MB/s eta 0:00:00


In [2]:
import asyncio
import nest_asyncio
from ddgs import DDGS
import pandas as pd
import re
from datetime import datetime
import time
import random
import json
from urllib.parse import urlparse
import aiohttp
from typing import List, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# Enable nested asyncio event loops for Jupyter
nest_asyncio.apply()

class ComprehensiveNigerianStudentScraper:
    def __init__(self):
        # All European countries (EU + EEA + UK + other European countries)
        self.COUNTRIES = [
            # EU Countries
            'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic',
            'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece',
            'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
            'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia',
            'Slovenia', 'Spain', 'Sweden',
            # Non-EU European Countries
            'United Kingdom', 'Norway', 'Switzerland', 'Iceland', 'Liechtenstein',
            'Albania', 'Andorra', 'Armenia', 'Azerbaijan', 'Belarus', 'Bosnia and Herzegovina',
            'Georgia', 'Kazakhstan', 'Kosovo', 'Moldova', 'Monaco', 'Montenegro',
            'North Macedonia', 'Russia', 'San Marino', 'Serbia', 'Turkey', 'Ukraine',
            'Vatican City'
        ]

        # Comprehensive universities by country
        self.UNIVERSITIES = {
            'Germany': [
                'TU Munich', 'University of Munich (LMU)', 'Heidelberg University', 'Humboldt University Berlin',
                'RWTH Aachen', 'University of Hamburg', 'Free University Berlin', 'University of Cologne',
                'University of Frankfurt', 'University of Göttingen', 'University of Bonn', 'TU Berlin',
                'University of Stuttgart', 'University of Würzburg', 'University of Freiburg',
                'University of Mannheim', 'Karlsruhe Institute of Technology', 'TU Dresden',
                'University of Leipzig', 'University of Münster', 'University of Erlangen-Nuremberg'
            ],
            'France': [
                'Sorbonne University', 'Ecole Polytechnique', 'Sciences Po', 'HEC Paris', 'INSEAD',
                'University of Paris', 'Ecole Normale Supérieure', 'University of Strasbourg',
                'University of Lyon', 'University of Toulouse', 'University of Marseille',
                'University of Bordeaux', 'University of Lille', 'University of Montpellier',
                'University of Nantes', 'University of Rennes', 'CentraleSupélec', 'ESSEC',
                'University of Grenoble', 'University of Nice'
            ],
            'Netherlands': [
                'University of Amsterdam', 'Delft University of Technology', 'Leiden University',
                'Utrecht University', 'Erasmus University Rotterdam', 'University of Groningen',
                'Eindhoven University of Technology', 'VU Amsterdam', 'Wageningen University',
                'Tilburg University', 'Maastricht University', 'University of Twente',
                'Radboud University', 'Rotterdam University of Applied Sciences'
            ],
            'United Kingdom': [
                'Oxford University', 'Cambridge University', 'Imperial College London', 'UCL',
                'King\'s College London', 'LSE', 'University of Edinburgh', 'University of Manchester',
                'University of Warwick', 'University of Bristol', 'University of Glasgow',
                'University of Birmingham', 'University of Leeds', 'University of Southampton',
                'University of Nottingham', 'University of Sheffield', 'University of Liverpool',
                'University of York', 'Durham University', 'University of St Andrews',
                'Queen Mary University', 'Cardiff University', 'University of Bath'
            ],
            'Italy': [
                'Bocconi University', 'University of Bologna', 'Sapienza University of Rome',
                'Politecnico di Milano', 'University of Milan', 'University of Florence',
                'University of Turin', 'University of Naples', 'University of Padua',
                'University of Pisa', 'University of Rome Tor Vergata', 'University of Genoa',
                'University of Pavia', 'University of Venice', 'University of Siena'
            ],
            'Spain': [
                'IE University', 'ESADE', 'Universidad Complutense Madrid', 'Universidad Autónoma de Madrid',
                'Universidad de Barcelona', 'Universidad Politécnica de Madrid', 'Universidad de Valencia',
                'Universidad de Sevilla', 'Universidad del País Vasco', 'Universidad de Granada',
                'Universidad de Zaragoza', 'Universidad de Salamanca', 'Universitat Pompeu Fabra'
            ],
            'Poland': [
                'University of Warsaw', 'Jagiellonian University', 'Warsaw University of Technology',
                'AGH University of Science and Technology', 'University of Krakow',
                'Gdansk University of Technology', 'University of Wrocław', 'Adam Mickiewicz University',
                'Nicolaus Copernicus University', 'University of Lodz'
            ],
            'Sweden': [
                'KTH Royal Institute of Technology', 'Lund University', 'Stockholm University',
                'University of Gothenburg', 'Uppsala University', 'Chalmers University of Technology',
                'Linköping University', 'Malmö University', 'Umeå University'
            ],
            'Norway': [
                'University of Oslo', 'Norwegian University of Science and Technology',
                'University of Bergen', 'Norwegian School of Economics', 'University of Tromsø',
                'Norwegian University of Life Sciences', 'University of Stavanger'
            ],
            'Denmark': [
                'University of Copenhagen', 'Technical University of Denmark', 'Aarhus University',
                'Copenhagen Business School', 'Aalborg University', 'University of Southern Denmark'
            ],
            'Finland': [
                'University of Helsinki', 'Aalto University', 'University of Turku',
                'Tampere University', 'University of Oulu', 'University of Jyväskylä'
            ],
            'Belgium': [
                'KU Leuven', 'Ghent University', 'Université libre de Bruxelles',
                'University of Antwerp', 'Université catholique de Louvain', 'Vrije Universiteit Brussel'
            ],
            'Austria': [
                'University of Vienna', 'Vienna University of Technology', 'University of Graz',
                'University of Innsbruck', 'Vienna University of Economics and Business',
                'University of Salzburg', 'Johannes Kepler University Linz'
            ],
            'Switzerland': [
                'ETH Zurich', 'University of Zurich', 'EPFL', 'University of Geneva',
                'University of Basel', 'University of Bern', 'University of Lausanne',
                'University of St. Gallen'
            ],
            'Ireland': [
                'Trinity College Dublin', 'University College Dublin', 'University College Cork',
                'National University of Ireland Galway', 'Dublin City University',
                'Maynooth University', 'University of Limerick'
            ],
            'Portugal': [
                'University of Porto', 'University of Lisbon', 'University of Coimbra',
                'Nova University of Lisbon', 'University of Aveiro', 'University of Minho'
            ],
            'Czech Republic': [
                'Charles University', 'Czech Technical University', 'Masaryk University',
                'Brno University of Technology', 'University of Economics Prague'
            ],
            'Hungary': [
                'Eötvös Loránd University', 'Budapest University of Technology',
                'University of Szeged', 'University of Debrecen', 'Corvinus University'
            ],
            'Romania': [
                'University of Bucharest', 'Babeș-Bolyai University', 'Alexandru Ioan Cuza University',
                'Polytechnic University of Bucharest', 'West University of Timișoara'
            ],
            'Greece': [
                'National and Kapodistrian University of Athens', 'Aristotle University of Thessaloniki',
                'National Technical University of Athens', 'University of Crete', 'University of Patras'
            ],
            'Croatia': [
                'University of Zagreb', 'University of Split', 'University of Rijeka',
                'University of Osijek', 'Josip Juraj Strossmayer University'
            ],
            'Slovenia': [
                'University of Ljubljana', 'University of Maribor', 'University of Primorska'
            ],
            'Slovakia': [
                'Comenius University', 'Slovak University of Technology', 'University of Economics'
            ],
            'Estonia': [
                'University of Tartu', 'Tallinn University of Technology', 'Tallinn University'
            ],
            'Latvia': [
                'University of Latvia', 'Riga Technical University', 'Latvia University of Agriculture'
            ],
            'Lithuania': [
                'Vilnius University', 'Kaunas University of Technology', 'Lithuanian University of Health Sciences'
            ],
            'Bulgaria': [
                'Sofia University', 'Technical University of Sofia', 'University of Plovdiv'
            ],
            'Cyprus': [
                'University of Cyprus', 'Cyprus University of Technology', 'European University Cyprus'
            ],
            'Malta': [
                'University of Malta', 'Malta College of Arts Science and Technology'
            ],
            'Luxembourg': [
                'University of Luxembourg'
            ],
            'Iceland': [
                'University of Iceland', 'Reykjavik University'
            ],
            'Russia': [
                'Moscow State University', 'Saint Petersburg State University', 'MIPT',
                'Higher School of Economics', 'Bauman Moscow State Technical University'
            ],
            'Turkey': [
                'Boğaziçi University', 'Middle East Technical University', 'Istanbul Technical University',
                'Bilkent University', 'Istanbul University', 'Koç University'
            ],
            'Ukraine': [
                'Taras Shevchenko National University', 'Igor Sikorsky Kyiv Polytechnic Institute',
                'V. N. Karazin Kharkiv National University'
            ]
        }

        # Polytechnics and Technical Colleges
        self.POLYTECHNICS = {
            'Germany': [
                'FH Aachen', 'Hamburg University of Applied Sciences', 'Frankfurt University of Applied Sciences',
                'Munich University of Applied Sciences', 'Cologne University of Applied Sciences'
            ],
            'Netherlands': [
                'Amsterdam University of Applied Sciences', 'The Hague University of Applied Sciences',
                'Rotterdam University of Applied Sciences', 'Fontys University of Applied Sciences'
            ],
            'United Kingdom': [
                'Imperial College London', 'University of Greenwich', 'Coventry University',
                'Teesside University', 'University of Hertfordshire'
            ],
            'France': [
                'Université de technologie de Compiègne', 'Université de technologie de Troyes',
                'Institut national des sciences appliquées'
            ],
            'Italy': [
                'Politecnico di Milano', 'Politecnico di Torino', 'Politecnico di Bari'
            ],
            'Spain': [
                'Universidad Politécnica de Madrid', 'Universidad Politécnica de Valencia',
                'Universidad Politécnica de Cataluña'
            ],
            'Poland': [
                'Warsaw University of Technology', 'Wrocław University of Science and Technology',
                'AGH University of Science and Technology'
            ],
            'Portugal': [
                'Instituto Politécnico de Lisboa', 'Instituto Politécnico do Porto',
                'Instituto Politécnico de Coimbra'
            ]
        }

        # Colleges of Education
        self.COLLEGES_OF_EDUCATION = {
            'United Kingdom': [
                'Institute of Education UCL', 'King\'s College London School of Education',
                'University of Cambridge Faculty of Education', 'Oxford Department of Education'
            ],
            'Germany': [
                'Pädagogische Hochschule Heidelberg', 'Pädagogische Hochschule Freiburg',
                'University of Education Weingarten'
            ],
            'Netherlands': [
                'Hogeschool van Amsterdam', 'Fontys Teacher Training College',
                'HU University of Applied Sciences Utrecht'
            ],
            'France': [
                'ESPÉ (Écoles supérieures du professorat)', 'INSPE', 'École normale supérieure'
            ],
            'Ireland': [
                'Mary Immaculate College', 'Church of Ireland College of Education',
                'Marino Institute of Education'
            ],
            'Sweden': [
                'Stockholm Institute of Education', 'Malmö University Faculty of Education'
            ]
        }

        self.MAX_RETRIES = 3
        self.SEARCH_DELAY = 2
        self.results = []

        # Generate comprehensive search keywords
        self.keywords = self.generate_comprehensive_keywords()

        print(f"Initialized comprehensive scraper with {len(self.keywords)} search queries")
        print(f"Covering {len(self.COUNTRIES)} countries")
        print(f"Including {sum(len(unis) for unis in self.UNIVERSITIES.values())} universities")
        print(f"Including {sum(len(polys) for polys in self.POLYTECHNICS.values())} polytechnics")
        print(f"Including {sum(len(coes) for coes in self.COLLEGES_OF_EDUCATION.values())} colleges of education")

    def generate_comprehensive_keywords(self) -> List[str]:
        """Generate comprehensive search keywords for all platforms and institutions"""
        keywords = []

        # Nigerian student search terms
        nigerian_terms = [
            'Nigerian students', 'Nigeria students', 'Nigerians in',
            'Nigerian association', 'Nigerian society', 'NANS',
            'Nigerian Student Association', 'NSA', 'Nigerian Union',
            'Naija students', 'Nigerian community'
        ]

        # Social media platforms
        platforms = [
            'site:facebook.com/groups',
            'site:facebook.com/pages',
            'site:linkedin.com/groups',
            'site:linkedin.com/company',
            'site:t.me',
            'site:discord.gg',
            'site:reddit.com/r',
            'site:whatsapp.com',
            'site:twitter.com',
            'site:instagram.com',
            'site:youtube.com'
        ]

        # Institution types
        institution_types = ['university', 'polytechnic', 'college', 'institute', 'school']

        # Generate keywords for all countries
        for country in self.COUNTRIES:
            # Basic country searches
            for platform in platforms:
                for term in nigerian_terms[:5]:  # Limit terms per country
                    keywords.append(f'{platform} "{term}" "{country}"')
                    for inst_type in institution_types[:3]:  # Limit institution types
                        keywords.append(f'{platform} "{term}" "{country}" {inst_type}')

            # University-specific searches
            if country in self.UNIVERSITIES:
                for university in self.UNIVERSITIES[country][:5]:  # Top 5 universities per country
                    for platform in platforms[:4]:  # Top 4 platforms
                        for term in nigerian_terms[:3]:  # Top 3 terms
                            keywords.append(f'{platform} "{term}" "{university}"')

            # Polytechnic-specific searches
            if country in self.POLYTECHNICS:
                for polytechnic in self.POLYTECHNICS[country][:3]:  # Top 3 polytechnics
                    for platform in platforms[:3]:  # Top 3 platforms
                        keywords.append(f'{platform} "Nigerian students" "{polytechnic}"')

            # College of Education searches
            if country in self.COLLEGES_OF_EDUCATION:
                for college in self.COLLEGES_OF_EDUCATION[country][:3]:  # Top 3 colleges
                    for platform in platforms[:3]:  # Top 3 platforms
                        keywords.append(f'{platform} "Nigerian students" "{college}"')

        # Remove duplicates and shuffle
        keywords = list(set(keywords))
        random.shuffle(keywords)

        return keywords[:200]  # Limit to 200 queries for reasonable execution time

    def clean_link(self, link: str) -> str:
        """Clean and normalize links"""
        return re.sub(r"\?.*$", "", link)

    def extract_country_from_text(self, text: str) -> str:
        """Extract country from text content"""
        text_lower = text.lower()
        for country in self.COUNTRIES:
            if country.lower() in text_lower:
                return country
        return "Unknown"

    def extract_institution_from_text(self, text: str) -> Dict[str, str]:
        """Extract institution details from text content"""
        text_lower = text.lower()

        # Check universities
        for country, universities in self.UNIVERSITIES.items():
            for university in universities:
                if university.lower() in text_lower:
                    return {
                        "institution_name": university,
                        "institution_type": "University",
                        "country": country
                    }

        # Check polytechnics
        for country, polytechnics in self.POLYTECHNICS.items():
            for polytechnic in polytechnics:
                if polytechnic.lower() in text_lower:
                    return {
                        "institution_name": polytechnic,
                        "institution_type": "Polytechnic",
                        "country": country
                    }

        # Check colleges of education
        for country, colleges in self.COLLEGES_OF_EDUCATION.items():
            for college in colleges:
                if college.lower() in text_lower:
                    return {
                        "institution_name": college,
                        "institution_type": "College of Education",
                        "country": country
                    }

        # Check for general institution patterns
        institution_patterns = [
            (r'university of (\w+)', "University"),
            (r'(\w+) university', "University"),
            (r'(\w+) college', "College"),
            (r'(\w+) polytechnic', "Polytechnic"),
            (r'(\w+) institute', "Institute")
        ]

        for pattern, inst_type in institution_patterns:
            match = re.search(pattern, text_lower)
            if match:
                return {
                    "institution_name": match.group(0).title(),
                    "institution_type": inst_type,
                    "country": "Unknown"
                }

        return {
            "institution_name": "Unknown Institution",
            "institution_type": "Unknown",
            "country": "Unknown"
        }

    def extract_session_from_text(self, text: str) -> str:
        """Extract academic session from text"""
        session_patterns = [
            r'20\d{2}[/-]20\d{2}',
            r'20\d{2}/\d{2}',
            r'academic year 20\d{2}',
            r'session 20\d{2}'
        ]

        for pattern in session_patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                return match.group()

        return "Unknown"

    def identify_platform(self, url: str) -> str:
        """Identify social media platform from URL"""
        url_lower = url.lower()

        platform_mapping = {
            "facebook.com": "Facebook",
            "linkedin.com": "LinkedIn",
            "t.me": "Telegram",
            "telegram": "Telegram",
            "discord": "Discord",
            "reddit.com": "Reddit",
            "whatsapp": "WhatsApp",
            "twitter.com": "Twitter",
            "instagram.com": "Instagram",
            "youtube.com": "YouTube",
            "tiktok.com": "TikTok"
        }

        for domain, platform in platform_mapping.items():
            if domain in url_lower:
                return platform

        return "Unknown"

    def extract_member_count_from_text(self, text: str) -> str:
        """Extract member count using regex patterns"""
        patterns = [
            r'([\d,]+)\s+members?',
            r'([\d,]+)\s+people',
            r'([\d,]+)\s+followers',
            r'([\d,]+)\s+participants',
            r'([\d,]+)\s+subscribers',
            r'([\d,]+)\s+students',
            r'([\d,]+)\s+users'
        ]

        for pattern in patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                return match.group(1).replace(',', '')

        return "Unknown"

    async def extract_comprehensive_metadata(self, url: str, title: str, snippet: str) -> Dict:
        """Extract comprehensive metadata without browser automation"""

        platform = self.identify_platform(url)

        # Combine title and snippet for analysis
        full_text = f"{title} {snippet}"

        country = self.extract_country_from_text(full_text)
        institution_info = self.extract_institution_from_text(full_text)
        session = self.extract_session_from_text(full_text)
        members_count = self.extract_member_count_from_text(full_text)

        # Clean group name from title
        group_name = title.strip() if title else "Unknown Group"
        # Remove common suffixes
        group_name = re.sub(r'\s*-\s*(Facebook|LinkedIn|Telegram|Discord|Reddit|Twitter|Instagram|YouTube).*$',
                          '', group_name, flags=re.IGNORECASE)

        return {
            "Date": datetime.utcnow().strftime("%Y-%m-%d"),
            "Day": datetime.utcnow().strftime("%A"),
            "Platform": platform,
            "Group/Page Name": group_name,
            "Country": country if country != "Unknown" else institution_info["country"],
            "Institution Name": institution_info["institution_name"],
            "Institution Type": institution_info["institution_type"],
            "Academic Session": session,
            "Members Count": members_count,
            "Social Media Link": self.clean_link(url),
            "Description": snippet[:300] if snippet else "",
            "Scraped At": datetime.utcnow().isoformat(),
            "Search Query": "",  # Will be filled during search
            "Relevance Score": self.calculate_relevance_score(full_text)
        }

    def calculate_relevance_score(self, text: str) -> int:
        """Calculate relevance score based on keywords"""
        text_lower = text.lower()
        score = 0

        # Nigerian-related keywords
        nigerian_keywords = ['nigerian', 'nigeria', 'naija', 'nans']
        for keyword in nigerian_keywords:
            if keyword in text_lower:
                score += 2

        # Student-related keywords
        student_keywords = ['student', 'association', 'union', 'society', 'community']
        for keyword in student_keywords:
            if keyword in text_lower:
                score += 1

        # Institution keywords
        institution_keywords = ['university', 'polytechnic', 'college', 'institute', 'school']
        for keyword in institution_keywords:
            if keyword in text_lower:
                score += 1

        return score

    async def search_with_retry(self, query: str, max_retries: int = 3) -> List[Dict]:
        """Search with retry logic using DDGS"""
        for attempt in range(max_retries):
            try:
                print(f"Searching: {query[:60]}... (attempt {attempt + 1})")

                # Use DDGS for search
                search_results = []
                with DDGS() as ddgs:
                    for result in ddgs.text(query, max_results=10):
                        search_results.append(result)

                print(f"Found {len(search_results)} results")
                return search_results

            except Exception as e:
                print(f"Search attempt {attempt + 1} failed: {str(e)[:100]}")
                if attempt < max_retries - 1:
                    delay = (attempt + 1) * 3
                    print(f"Waiting {delay} seconds before retry...")
                    await asyncio.sleep(delay)
                else:
                    print(f"All search attempts failed for query")
                    return []

    async def run_comprehensive_scraping(self):
        """Main comprehensive scraping function"""
        print("Starting Comprehensive European Nigerian Student Groups Scraper...")
        print(f"Will process {len(self.keywords)} search queries")
        print(f"Targeting {len(self.COUNTRIES)} European countries")

        for i, query in enumerate(self.keywords, 1):
            print(f"\nProcessing query {i}/{len(self.keywords)}")

            try:
                search_results = await self.search_with_retry(query)

                if not search_results:
                    print("No results found for this query")
                    continue

                for j, result in enumerate(search_results, 1):
                    url = result.get("href", "")
                    title = result.get("title", "")
                    snippet = result.get("body", "")

                    # Filter for relevant social media URLs
                    social_media_domains = [
                        "facebook", "linkedin", "telegram", "t.me", "discord",
                        "reddit", "whatsapp", "twitter", "instagram", "youtube", "tiktok"
                    ]

                    if not url or not any(domain in url.lower() for domain in social_media_domains):
                        continue

                    print(f"Processing URL {j}: {url[:60]}...")

                    try:
                        metadata = await self.extract_comprehensive_metadata(url, title, snippet)
                        metadata["Search Query"] = query

                        # Only add if we found a European country or relevant institution
                        if (metadata["Country"] != "Unknown" or
                            metadata["Institution Name"] != "Unknown Institution" or
                            metadata["Relevance Score"] >= 3):

                            self.results.append(metadata)
                            print(f"Added: {metadata['Group/Page Name'][:30]}... - {metadata['Country']} - {metadata['Institution Type']}")
                        else:
                            print("Skipped: No European relevance found")

                    except Exception as e:
                        print(f"Error processing URL: {str(e)[:50]}")
                        continue

                # Delay between queries
                if i < len(self.keywords):
                    delay = random.uniform(1, 3)  # Random delay
                    print(f"Waiting {delay:.1f} seconds...")
                    await asyncio.sleep(delay)

            except Exception as e:
                print(f"Error processing query: {str(e)[:100]}")
                continue

        print(f"\nScraping completed! Found {len(self.results)} relevant groups/pages")
        return self.results

    def save_comprehensive_results(self, filename: str = "comprehensive_nigerian_student_groups_europe.xlsx"):
        """Save comprehensive results to Excel file with multiple organized sheets"""
        if not self.results:
            print("No results to save")
            return None

        df = pd.DataFrame(self.results)

        # Remove duplicates based on source URL
        df = df.drop_duplicates(subset=['Social Media Link'], keep='first')

        # Sort by relevance score and country
        df = df.sort_values(['Relevance Score', 'Country', 'Platform'], ascending=[False, True, True])

        # Save to Excel with multiple organized sheets
        try:
            with pd.ExcelWriter(filename, engine='openpyxl') as writer:
                # Main comprehensive sheet
                df.to_excel(writer, sheet_name='All Groups & Pages', index=False)

                # Platform-wise sheets
                for platform in sorted(df['Platform'].unique()):
                    if platform != 'Unknown':
                        platform_data = df[df['Platform'] == platform]
                        sheet_name = f"{platform}_Groups"[:30]
                        platform_data.to_excel(writer, sheet_name=sheet_name, index=False)

                # Country-wise sheets (top countries only)
                country_counts = df['Country'].value_counts()
                top_countries = country_counts.head(10).index

                for country in top_countries:
                    if country != 'Unknown':
                        country_data = df[df['Country'] == country]
                        sheet_name = f"{country}_Students"[:30]
                        country_data.to_excel(writer, sheet_name=sheet_name, index=False)

                # Institution type sheets
                for inst_type in sorted(df['Institution Type'].unique()):
                    if inst_type != 'Unknown':
                        inst_data = df[df['Institution Type'] == inst_type]
                        sheet_name = f"{inst_type.replace(' ', '_')}"[:30]
                        inst_data.to_excel(writer, sheet_name=sheet_name, index=False)

                # High relevance sheet (score >= 4)
                high_relevance = df[df['Relevance Score'] >= 4]
                if not high_relevance.empty:
                    high_relevance.to_excel(writer, sheet_name='High_Relevance', index=False)

                # Summary statistics sheet
                summary_data = []

                # Country summary
                country_summary = df.groupby('Country').agg({
                    'Group/Page Name': 'count',
                    'Platform': lambda x: ', '.join(x.unique()[:3]),
                    'Institution Type': lambda x: ', '.join(x.unique()[:3]),
                    'Relevance Score': 'mean'
                }).reset_index()
                country_summary.columns = ['Country', 'Group_Count', 'Top_Platforms', 'Institution_Types', 'Avg_Relevance']
                country_summary.to_excel(writer, sheet_name='Country_Summary', index=False)

                # Platform summary
                platform_summary = df.groupby('Platform').agg({
                    'Group/Page Name': 'count',
                    'Country': lambda x: len(x.unique()),
                    'Relevance Score': 'mean'
                }).reset_index()
                platform_summary.columns = ['Platform', 'Group_Count', 'Countries_Covered', 'Avg_Relevance']
                platform_summary.to_excel(writer, sheet_name='Platform_Summary', index=False)

                # Institution type summary
                institution_summary = df.groupby('Institution Type').agg({
                    'Group/Page Name': 'count',
                    'Country': lambda x: len(x.unique()),
                    'Platform': lambda x: ', '.join(x.unique()[:3])
                }).reset_index()
                institution_summary.columns = ['Institution_Type', 'Group_Count', 'Countries', 'Top_Platforms']
                institution_summary.to_excel(writer, sheet_name='Institution_Summary', index=False)

            print(f"Comprehensive results saved to {filename}")

        except Exception as e:
            print(f"Error saving Excel file: {e}")

        # Also save as CSV
        try:
            csv_filename = filename.replace('.xlsx', '.csv')
            df.to_csv(csv_filename, index=False, encoding='utf-8')
            print(f"Results also saved to {csv_filename}")
        except Exception as e:
            print(f"Error saving CSV file: {e}")

        # Print comprehensive summary
        self.print_comprehensive_summary(df)

        return df

    def print_comprehensive_summary(self, df: pd.DataFrame):
        """Print comprehensive scraping summary"""
        print(f"\n{'='*70}")
        print("COMPREHENSIVE SCRAPING SUMMARY")
        print(f"{'='*70}")
        print(f"Total groups/pages found: {len(df)}")
        print(f"Unique countries covered: {df['Country'].nunique()}")
        print(f"Platforms covered: {', '.join(sorted(df['Platform'].unique()))}")
        print(f"Institution types found: {', '.join(sorted(df['Institution Type'].unique()))}")

        print(f"\nTOP 10 COUNTRIES BY GROUP COUNT:")
        country_counts = df['Country'].value_counts().head(10)
        for i, (country, count) in enumerate(country_counts.items(), 1):
            print(f"  {i:2d}. {country}: {count} groups/pages")

        print(f"\nPLATFORM DISTRIBUTION:")
        platform_counts = df['Platform'].value_counts()
        for platform, count in platform_counts.items():
            print(f"  {platform}: {count} groups/pages")

        print(f"\nINSTITUTION TYPE DISTRIBUTION:")
        institution_counts = df['Institution Type'].value_counts()
        for inst_type, count in institution_counts.items():
            print(f"  {inst_type}: {count} groups/pages")

        print(f"\nRELEVANCE SCORE DISTRIBUTION:")
        relevance_counts = df['Relevance Score'].value_counts().sort_index(ascending=False)
        for score, count in relevance_counts.items():
            print(f"  Score {score}: {count} groups/pages")

        print(f"\nTOP UNIVERSITIES WITH NIGERIAN STUDENT GROUPS:")
        university_groups = df[df['Institution Type'] == 'University']['Institution Name'].value_counts().head(10)
        for i, (university, count) in enumerate(university_groups.items(), 1):
            if university != 'Unknown Institution':
                print(f"  {i:2d}. {university}: {count} groups")

        print(f"\nTOP POLYTECHNICS WITH NIGERIAN STUDENT GROUPS:")
        polytechnic_groups = df[df['Institution Type'] == 'Polytechnic']['Institution Name'].value_counts().head(5)
        for i, (polytechnic, count) in enumerate(polytechnic_groups.items(), 1):
            if polytechnic != 'Unknown Institution':
                print(f"  {i}. {polytechnic}: {count} groups")

        print(f"\nSAMPLE HIGH-RELEVANCE GROUPS FOUND:")
        high_relevance = df[df['Relevance Score'] >= 4].head(5)
        for i, row in high_relevance.iterrows():
            print(f"  - {row['Group/Page Name'][:50]}...")
            print(f"    Platform: {row['Platform']}, Country: {row['Country']}")
            print(f"    Institution: {row['Institution Name'][:40]}... ({row['Institution Type']})")
            print(f"    Link: {row['Social Media Link'][:60]}...")
            print()

        print(f"{'='*70}")

# Main execution functions
async def run_comprehensive_scraper_async():
    """Run the comprehensive scraper asynchronously"""
    scraper = ComprehensiveNigerianStudentScraper()

    # Run the comprehensive scraping
    results = await scraper.run_comprehensive_scraping()

    # Save results
    if results:
        df = scraper.save_comprehensive_results()
        return df
    else:
        print("No results found. Try different search terms or check internet connection.")
        return None

def run_comprehensive_scraper():
    """Run the comprehensive scraper in Jupyter notebook"""
    try:
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(run_comprehensive_scraper_async())
    except Exception as e:
        print(f"Error running comprehensive scraper: {e}")
        return None

# Additional utility functions for data analysis
def analyze_results(df):
    """Analyze the scraped results"""
    if df is None or df.empty:
        print("No data to analyze")
        return

    print("DETAILED ANALYSIS OF SCRAPED DATA")
    print("="*50)

    # Geographic distribution
    print("\n1. GEOGRAPHIC DISTRIBUTION:")
    europe_regions = {
        'Western Europe': ['Germany', 'France', 'Netherlands', 'Belgium', 'Austria', 'Switzerland', 'Luxembourg'],
        'Northern Europe': ['United Kingdom', 'Sweden', 'Norway', 'Denmark', 'Finland', 'Iceland'],
        'Southern Europe': ['Italy', 'Spain', 'Portugal', 'Greece', 'Malta', 'Cyprus'],
        'Eastern Europe': ['Poland', 'Czech Republic', 'Hungary', 'Romania', 'Bulgaria', 'Slovakia', 'Slovenia', 'Croatia'],
        'Baltic States': ['Estonia', 'Latvia', 'Lithuania'],
        'Other Europe': ['Russia', 'Turkey', 'Ukraine', 'Belarus', 'Serbia', 'Bosnia and Herzegovina']
    }

    for region, countries in europe_regions.items():
        region_count = df[df['Country'].isin(countries)]['Group/Page Name'].count()
        if region_count > 0:
            print(f"  {region}: {region_count} groups")

    # Platform effectiveness
    print("\n2. PLATFORM EFFECTIVENESS:")
    platform_relevance = df.groupby('Platform')['Relevance Score'].agg(['count', 'mean']).round(2)
    platform_relevance.columns = ['Group_Count', 'Avg_Relevance']
    platform_relevance = platform_relevance.sort_values('Avg_Relevance', ascending=False)
    print(platform_relevance)

    # Institution analysis
    print("\n3. INSTITUTION TYPE ANALYSIS:")
    institution_analysis = df.groupby('Institution Type').agg({
        'Group/Page Name': 'count',
        'Country': lambda x: len(x.unique()),
        'Relevance Score': 'mean'
    }).round(2)
    institution_analysis.columns = ['Groups', 'Countries', 'Avg_Relevance']
    print(institution_analysis)

def export_links_table(df, filename="nigerian_student_links_table.xlsx"):
    """Export a clean table of just the links organized by country and institution type"""
    if df is None or df.empty:
        print("No data to export")
        return

    # Create a clean links table
    links_table = df[['Country', 'Institution Type', 'Institution Name', 'Platform',
                     'Group/Page Name', 'Social Media Link', 'Members Count', 'Relevance Score']].copy()

    # Sort by country, institution type, and relevance
    links_table = links_table.sort_values(['Country', 'Institution Type', 'Relevance Score'],
                                        ascending=[True, True, False])

    try:
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            # Main links table
            links_table.to_excel(writer, sheet_name='All_Links', index=False)

            # Separate sheets for each institution type
            for inst_type in ['University', 'Polytechnic', 'College of Education']:
                inst_links = links_table[links_table['Institution Type'] == inst_type]
                if not inst_links.empty:
                    sheet_name = inst_type.replace(' ', '_')[:30]
                    inst_links.to_excel(writer, sheet_name=sheet_name, index=False)

            # High-quality links only (relevance >= 4)
            high_quality = links_table[links_table['Relevance Score'] >= 4]
            if not high_quality.empty:
                high_quality.to_excel(writer, sheet_name='High_Quality_Links', index=False)

        print(f"Links table exported to {filename}")

    except Exception as e:
        print(f"Error exporting links table: {e}")

# Display comprehensive usage instructions
def display_usage_instructions():
    """Display usage instructions for the scraper"""
    print("=" * 80)
    print("COMPREHENSIVE EUROPEAN NIGERIAN STUDENT GROUPS SCRAPER")
    print("=" * 80)
    print("This comprehensive scraper covers:")
    print("• All European countries (EU + EEA + UK + other European nations)")
    print("• Universities, Polytechnics, and Colleges of Education")
    print("• Multiple social media platforms (Facebook, LinkedIn, Telegram, etc.)")
    print("• Advanced relevance scoring and filtering")
    print()
    print("USAGE INSTRUCTIONS:")
    print("1. Run comprehensive scraper:")
    print("   df = run_comprehensive_scraper()")
    print()
    print("2. Analyze results:")
    print("   analyze_results(df)")
    print()
    print("3. Export clean links table:")
    print("   export_links_table(df)")
    print()
    print("4. Access specific data:")
    print("   # High relevance groups only")
    print("   high_relevance = df[df['Relevance Score'] >= 4]")
    print()
    print("   # Specific country groups")
    print("   germany_groups = df[df['Country'] == 'Germany']")
    print()
    print("   # Specific platform groups")
    print("   facebook_groups = df[df['Platform'] == 'Facebook']")
    print()
    print("   # University groups only")
    print("   university_groups = df[df['Institution Type'] == 'University']")
    print("=" * 80)
    print("Note: The scraper will create multiple Excel sheets organized by:")
    print("- Country, Platform, Institution Type")
    print("- Summary statistics and analysis")
    print("- High-relevance groups for easy filtering")
    print("=" * 80)

# Call the function to display instructions
display_usage_instructions()

COMPREHENSIVE EUROPEAN NIGERIAN STUDENT GROUPS SCRAPER
This comprehensive scraper covers:
• All European countries (EU + EEA + UK + other European nations)
• Universities, Polytechnics, and Colleges of Education
• Multiple social media platforms (Facebook, LinkedIn, Telegram, etc.)
• Advanced relevance scoring and filtering

USAGE INSTRUCTIONS:
1. Run comprehensive scraper:
   df = run_comprehensive_scraper()

2. Analyze results:
   analyze_results(df)

3. Export clean links table:
   export_links_table(df)

4. Access specific data:
   # High relevance groups only
   high_relevance = df[df['Relevance Score'] >= 4]

   # Specific country groups
   germany_groups = df[df['Country'] == 'Germany']

   # Specific platform groups
   facebook_groups = df[df['Platform'] == 'Facebook']

   # University groups only
   university_groups = df[df['Institution Type'] == 'University']
Note: The scraper will create multiple Excel sheets organized by:
- Country, Platform, Institution Type
- Summary sta

In [3]:
display_usage_instructions()

COMPREHENSIVE EUROPEAN NIGERIAN STUDENT GROUPS SCRAPER
This comprehensive scraper covers:
• All European countries (EU + EEA + UK + other European nations)
• Universities, Polytechnics, and Colleges of Education
• Multiple social media platforms (Facebook, LinkedIn, Telegram, etc.)
• Advanced relevance scoring and filtering

USAGE INSTRUCTIONS:
1. Run comprehensive scraper:
   df = run_comprehensive_scraper()

2. Analyze results:
   analyze_results(df)

3. Export clean links table:
   export_links_table(df)

4. Access specific data:
   # High relevance groups only
   high_relevance = df[df['Relevance Score'] >= 4]

   # Specific country groups
   germany_groups = df[df['Country'] == 'Germany']

   # Specific platform groups
   facebook_groups = df[df['Platform'] == 'Facebook']

   # University groups only
   university_groups = df[df['Institution Type'] == 'University']
Note: The scraper will create multiple Excel sheets organized by:
- Country, Platform, Institution Type
- Summary sta

In [4]:
df = run_comprehensive_scraper()

Initialized comprehensive scraper with 200 search queries
Covering 51 countries
Including 243 universities
Including 29 polytechnics
Including 18 colleges of education
Starting Comprehensive European Nigerian Student Groups Scraper...
Will process 200 search queries
Targeting 51 European countries

Processing query 1/200
Searching: site:facebook.com/groups "Nigerians in" "University of Krako... (attempt 1)
Found 10 results
Processing URL 1: https://m.facebook.com/groups/nigeriansinnewyork/...
Added: Facebook Nigerians in New York... - Unknown - Unknown
Processing URL 2: https://www.facebook.com/ALKOM.Krakow/groups...
Skipped: No European relevance found
Processing URL 3: https://m.facebook.com/groups/965332910900317/posts/18170047...
Added: Facebook Nigerians in UK... - Unknown - Unknown
Processing URL 4: https://facebook.com/groups/tir.krakow...
Skipped: No European relevance found
Processing URL 5: https://www.facebook.com/groups/28621519152/...
Added: Facebook Robert Gordon Univers.